# Analyse exploratoire du marché Airbnb - Oslo (Version 2)
**Auteur :** Data Analyst (Assistant IA) & Business Analyst
**Contexte :** Projet Master 2

## 1. Introduction & Contexte métier
Ce notebook présente une **analyse exploratoire de données (EDA)** approfondie sur le marché Airbnb à Oslo. Il a pour but de fournir des insights clés à un investisseur potentiel ou à une collectivité locale.

Les **objectifs principaux** sont :
1. Comprendre la **structure de l'offre** (quartiers, types de logements).
2. Analyser les **prix** et identifier les zones les plus valorisées.
3. Étudier la **concurrence** (hôtes professionnels vs particuliers).
4. Estimer la **demande** via les avis et la disponibilité.
5. Proposer une **segmentation** du marché.

---
*Disclaimer : Cette analyse est basée sur une extraction statique (snapshot).*


## 2. Chargement des données
Nous commençons par importer les librairies nécessaires et charger le dataset.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

# Configuration du style des graphiques
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 14


In [ ]:
try:
    # Tentative chargement local
    df_raw = pd.read_csv('listings.csv')
    print("Données chargées avec succès (local) !")
except FileNotFoundError:
    try:
        # Fallback URL si local absent (pour portabilité)
        url = 'https://raw.githubusercontent.com/PercyaDJ/dossier-iae/refs/heads/main/listings.csv'
        df_raw = pd.read_csv(url)
        print("Données chargées avec succès (depuis GitHub) !")
    except:
        print("ERREUR : Fichier introuvable.")

print(f"Dimensions du dataset : {df_raw.shape}")


In [ ]:
df_raw.head(5)


## 3. Compréhension & Nettoyage des données
### 3.1. Infos générales et types


In [ ]:
df_raw.info()


### 3.2. Valeurs manquantes
Identifions les colonnes avec des `NaN`.


In [ ]:
missing_values = df_raw.isnull().sum()
missing_values[missing_values > 0]


**Note Importante sur les Prix :**
On constate que la colonne `price` contient un nombre significatif de valeurs manquantes (souvent 20-30% sur ce type de dataset). Ces lignes seront exclues de l'analyse tarifaire.

**Actions de nettoyage :**
1. Conversion de `last_review` en datetime.
2. Remplacer les NaN de `reviews_per_month` par 0 (absence d'avis).
3. Conservation d'un DataFrame de travail `df`.


In [ ]:
df = df_raw.copy()

if 'last_review' in df.columns:
    df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')

if 'reviews_per_month' in df.columns:
    df['reviews_per_month'] = df['reviews_per_month'].fillna(0)

print("Nettoyage basique terminé.")


### 3.3. Résumé statistique


In [ ]:
df.describe()


## 4. Vue d’ensemble du marché
### 4.1. Répartition par Quartier


In [ ]:
nb_counts = df['neighbourhood'].value_counts()

plt.figure(figsize=(12, 6))
sns.barplot(x=nb_counts.index, y=nb_counts.values, palette='viridis')
plt.xticks(rotation=45, ha='right')
plt.title('Nombre de listings par quartier à Oslo')
plt.show()


### 4.2. Répartition par Type de logement


In [ ]:
room_counts = df['room_type'].value_counts()

plt.figure(figsize=(8, 5))
sns.barplot(x=room_counts.index, y=room_counts.values, palette='magma')
plt.title('Répartition des types de logements')
plt.show()


## 5. Analyse des prix
### 5.1. Distribution des prix et Outliers
Visualisation de la distribution brute :


In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(df['price'].dropna(), bins=50, kde=True)
plt.title('Distribution des prix (données brutes)')
plt.xlabel('Prix (NOK)')
plt.show()


### 5.2. Stratégie de traitement des Outliers (CORRIGÉE)
Une première approche par IQR (Interquartile Range) standard supprime souvent trop de données sur les marchés immobiliers de luxe (jusqu'à 30% des données).

**Nouvelle Stratégie (Validée) :**
- Nous optons pour une suppression des **valeurs extrêmes (Top 1%)** uniquement.
- Nous supprimons également les prix nuls ou manquants.
- Cela permet de conserver le segment Luxe légitime tout en écartant les erreurs manifestes.


In [ ]:
# Calcul du seuil du 99ème percentile
upper_bound = df['price'].quantile(0.99)

print(f"Seuil supérieur (99%) : {upper_bound:.2f} NOK")

# Création de df_clean (Filtre : prix existant, > 0, et <= percentile 99)
df_clean = df[(df['price'] > 0) & (df['price'] <= upper_bound)].copy()

deleted_count = len(df) - len(df_clean)
deleted_pct = (deleted_count / len(df)) * 100

print(f"Nombre de listings avant : {len(df)}")
print(f"Nombre de listings après filtrage : {len(df_clean)}")
print(f"Données écartées (NaN ou Outliers) : {deleted_count} ({deleted_pct:.1f}%)")


### 5.3. Analyse des prix sur données nettoyées


In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(df_clean['price'], bins=50, kde=True, color='green')
plt.title(f'Distribution des prix (0 - {upper_bound:.0f} NOK)')
plt.xlabel('Prix (NOK)')
plt.show()


In [ ]:
plt.figure(figsize=(14, 7))
order_neigh = df_clean.groupby('neighbourhood')['price'].median().sort_values(ascending=False).index
sns.boxplot(data=df_clean, x='neighbourhood', y='price', order=order_neigh, palette='coolwarm')
plt.xticks(rotation=45, ha='right')
plt.title('Distribution des prix par quartier (Marché complet)')
plt.show()


## 6. Analyse des hôtes & des avis
### 6.1. Hôtes professionnels vs particuliers
Nous analysons si les hôtes "multi-listings" (>= 3 annonces) se comportent différemment.


In [ ]:
threshold_pro = 3
df_clean['is_pro'] = df_clean['calculated_host_listings_count'] >= threshold_pro

pro_counts = df_clean['is_pro'].value_counts(normalize=True) * 100

plt.figure(figsize=(6, 6))
pro_counts.plot.pie(autopct='%1.1f%%', labels=['Particuliers', 'Pros'], startangle=90, colors=['skyblue', 'salmon'])
plt.title('Part des hôtes Professionnels')
plt.show()


In [ ]:
# Comparaison des prix moyens
price_comparison = df_clean.groupby('is_pro')['price'].agg(['mean', 'median', 'count'])
price_comparison.index = ['Particulier', 'Professionnel']
print(price_comparison)


> **Insights Business (Hôtes - Analyse corrigée) :**
> - **Le Paradoxe des prix :** Contrairement à l'intuition, les hôtes professionnels pratiquent souvent des **prix moyens inférieurs** aux particuliers.
> - **Interprétation :** Les particuliers louent souvent leur résidence principale (bien équipés, grandes surfaces) pour amortir leurs charges, tandis que les professionnels optimisent le remplissage de biens plus petits et standardisés (studios, T2).


## 7. Indicateurs de demande
Le nombre d'avis est utilisé comme indicateur de volume de fréquentation.


In [ ]:
reviews_by_neigh = df_clean.groupby('neighbourhood')['number_of_reviews'].mean().sort_values(ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(x=reviews_by_neigh.index, y=reviews_by_neigh.values, palette='Blues_r')
plt.xticks(rotation=45, ha='right')
plt.title('Nombre moyen d\'avis par quartier')
plt.show()


## 8. Segmentation (Budget / Mid / Luxe)
Segmentation basée sur les terciles du marché nettoyé (incluant désormais le haut de gamme légitime).


In [ ]:
p33 = df_clean['price'].quantile(0.33)
p66 = df_clean['price'].quantile(0.66)

def segment_price(price):
    if price < p33: return 'Becom (Budget)'
    elif price < p66: return 'Mid-Range'
    else: return 'Premium/Luxe'

df_clean['segment'] = df_clean['price'].apply(segment_price)

print(f"Seuils de segmentation :")
print(f"- Budget (<{p33:.0f} NOK)")
print(f"- Luxe   (>{p66:.0f} NOK)")


In [ ]:
ct = pd.crosstab(df_clean['neighbourhood'], df_clean['segment'])
ct_norm = ct.div(ct.sum(1), axis=0)

ct_norm.plot(kind='bar', stacked=True, figsize=(12, 7), colormap='viridis')
plt.title('Segmentation de gamme par quartier')
plt.legend(bbox_to_anchor=(1, 1))
plt.show()


## 9. Synthèse business & limites

### 9.1. Insights Validés
1. **Marché** : Une offre concentrée sur le centre-ville et l'ouest d'Oslo.
2. **Prix** : L'ouest (Frogner) reste le plus cher, mais des quartiers comme Grünerløkka sont très dynamiques.
3. **Professionnels** : Ils représentent une part significative de l'offre mais proposent des biens en moyenne moins chers (standardisation), laissant le segment "Luxe/Authentique" aux particuliers.

### 9.2. Limites corrigées
- **Données manquantes** : ~30% des prix étaient manquants dans le jeu de données initial, ce qui réduit la base d'analyse.
- **Saisonnalité** : Données snapshot ne reflétant pas les variations été/hiver.


## 10. Fichiers à rendre
1. `oslo_airbnb_EDA.ipynb` (Ce notebook)
2. `interaction_IA_oslo.md` (Log d'interaction)
